In [1]:
%%capture output
!pip install --upgrade pip
!pip install --upgrade pandas
!pip install tables   
# necessary for pd.read_hdf()

!pip install ipywidgets
!pip install --upgrade jupyter
!pip install IProgress
!pip install catboost
!pip install shap
!pip install anndata

In [2]:
import os
import random
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, GroupKFold
import scipy
import anndata as ad
import shap

/dss/dsshome1/02/di93zoj/.local/lib/python3.8/site-packages/cupy/_environment.py:445: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


In [3]:
%matplotlib inline
from tqdm.notebook import tqdm
import gc
import pickle

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## data load

In [4]:
lrz_path = '/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/open-problems-multimodal-3rd-solution/'

model_path_for_now = '/dss/dsshome1/02/di93zoj/valentina/open-problems-multimodal-3rd-solution/'

raw_path =  lrz_path + 'input/raw/'  # '../../../input/raw/'

cite_target_path = lrz_path + 'input/target/cite/'   # '../../../input/target/cite/'
cite_feature_path = lrz_path + 'input/features/cite/'   # '../../../input/features/cite/'
cite_mlp_path = lrz_path + 'model/cite/mlp/'   # '../../../model/cite/mlp/'   # '../../../model/cite/mlp/'
cite_cb_path = lrz_path + 'model/cite/cb/'   # '../../../model/cite/cb/'

multi_target_path = lrz_path + 'input/target/multi/'   # '../../../input/target/multi/'
multi_feature_path = lrz_path + 'input/features/multi/'   # '../../../input/features/multi/'
multi_mlp_path = lrz_path + 'model/multi/mlp/'   # '../../../model/multi/mlp/'
multi_cb_path = lrz_path + 'model/multi/cb/'   # '../../../model/multi/cb/'

index_path = lrz_path + 'input/preprocess/cite/'

output_path = lrz_path + 'output/'   # '../../../output/'

## Cite

In [5]:
# get model name
#mlp_model_path = os.listdir(cite_mlp_path)

markdown

In [ ]:
# check model names and lists/dict/...

In [6]:
mlp_model_name = [
    'corr_add_con_imp',
    'corr_last_v3', 
    'corr_c_add_w2v_v1_mish_flg',
    'corr_c_add_w2v_v1_flg',
    'corr_c_add_84_v1',
    'corr_c_add_120_v1',
    'corr_w2v_cell_flg',
    'corr_best_cell_120',
    'corr_cluster_cell',
    'corr_w2v_128',
    'corr_imp_w2v_128',
    'corr_snorm',
    'corr_best_128',
    'corr_best_64',
    'corr_cluster_128',
    'corr_cluster_64',
    'corr_svd_128',
    'corr_svd_64',
             ]

In [7]:
model_name_list = []

for i in mlp_model_name:
    for num, j in enumerate(os.listdir(cite_mlp_path)):
        if i in j:
            model_name_list.append(j)

len(model_name_list)
model_name_list

['cite_mlp_corr_add_con_imp_flg_donor_val_50',
 'cite_mlp_corr_last_v3_flg_donor_val_55',
 'cite_mlp_corr_c_add_w2v_v1_mish_flg_donor_val_66',
 'cite_mlp_corr_c_add_w2v_v1_flg_donor_val_66',
 'cite_mlp_corr_c_add_84_v1_flg_donor_val_47',
 'cite_mlp_corr_c_add_120_v1_flg_donor_val_63',
 'cite_mlp_corr_w2v_cell_flg_donor_val_51',
 'cite_mlp_corr_best_cell_120_flg_donor_val_51',
 'cite_mlp_corr_cluster_cell_flg_donor_val_64',
 'cite_mlp_corr_w2v_128_flg_donor_val_42',
 'cite_mlp_corr_imp_w2v_128_flg_donor_val_38',
 'cite_mlp_corr_snorm_flg_donor_val_39',
 'cite_mlp_corr_best_128_flg_donor_val_45',
 'cite_mlp_corr_best_64_flg_donor_val_50',
 'cite_mlp_corr_cluster_128_flg_donor_val_51',
 'cite_mlp_corr_cluster_64_flg_donor_val_57',
 'cite_mlp_corr_svd_128_flg_donor_val_30',
 'cite_mlp_corr_svd_64_flg_donor_val_38']

In [8]:
weight = [1, 0.3, 1, 1, 1, 1, 1, 1, 1, 0.8, 0.8, 0.8, 0.8, 0.5, 0.5, 0.5, 1, 1, 2, 2]
weight_sum = np.array(weight).sum()
weight_sum

model_feat_dict = {model_name_list[0]:['X_test_add_con_imp.pickle', 1],
                   model_name_list[1]:['X_test_last_v3.pickle', 0.3],
                   model_name_list[2]:['X_test_c_add_w2v_v1.pickle', 1],
                   model_name_list[3]:['X_test_c_add_w2v_v1.pickle', 1],
                   model_name_list[4]:['X_test_c_add_84_v1.pickle', 1],
                   model_name_list[5]:['X_test_c_add_v1.pickle', 1],
                   
                   model_name_list[6]:['X_test_feature_w2v_cell.pickle', 1],
                   model_name_list[7]:['X_test_best_cell_128_120.pickle', 1],
                   model_name_list[8]:['X_test_cluster_cell_128.pickle', 1],
                   
                   model_name_list[9]:['X_test_feature_w2v.pickle', 0.8],
                   model_name_list[10]:['X_test_feature_imp_w2v.pickle',0.8],
                   model_name_list[11]:['X_test_feature_snorm.pickle', 0.8],
                   model_name_list[12]:['X_test_best_128.pickle', 0.8],
                   model_name_list[13]:['X_test_best_64.pickle', 0.5],
                   model_name_list[14]:['X_test_cluster_128.pickle', 0.5],
                   model_name_list[15]:['X_test_cluster_64.pickle', 0.5],
                   model_name_list[16]:['X_test_svd_128.pickle', 1],
                   model_name_list[17]:['X_test_svd_64.pickle', 1],
                   
                   'best_128':['X_test_best_128.pickle', 2],
                   'best_64':['X_test_best_64.pickle', 2],
                  }

### cite model

In [9]:
def std(x):
    x = np.array(x)
    return (x - x.mean(1).reshape(-1, 1)) / x.std(1).reshape(-1, 1)

In [10]:
class CiteDataset(Dataset):
    
    def __init__(self, feature, target):
        
        self.feature = feature
        self.target = target
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
                
        d = {
            "X": self.feature[index],
            "y" : self.target[index],
        }
        return d

In [11]:
class CiteDataset_test(Dataset):
    
    def __init__(self, feature):
        self.feature = feature
        
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self, index):
                
        d = {
            "X": self.feature[index]
        }
        return d

In [12]:
def partial_correlation_score_torch_faster(y_true, y_pred):
    """Compute the correlation between each rows of the y_true and y_pred tensors.
    Compatible with backpropagation.
    """
    y_true_centered = y_true - torch.mean(y_true, dim=1)[:,None]
    y_pred_centered = y_pred - torch.mean(y_pred, dim=1)[:,None]
    cov_tp = torch.sum(y_true_centered*y_pred_centered, dim=1)/(y_true.shape[1]-1)
    var_t = torch.sum(y_true_centered**2, dim=1)/(y_true.shape[1]-1)
    var_p = torch.sum(y_pred_centered**2, dim=1)/(y_true.shape[1]-1)
    return cov_tp/torch.sqrt(var_t*var_p)

def correl_loss(pred, tgt):
    """Loss for directly optimizing the correlation.
    """
    return -torch.mean(partial_correlation_score_torch_faster(tgt, pred))

In [13]:
class CiteModel(nn.Module):
    
    def __init__(self, feature_num):
        super(CiteModel, self).__init__()
        
        self.layer_seq_256 = nn.Sequential(nn.Linear(feature_num, 256),
                                           nn.Linear(256, 128),
                                       nn.LayerNorm(128),
                                       nn.ReLU(),
                                      )
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                       nn.Linear(64, 32),
                                       nn.LayerNorm(32),
                                       nn.ReLU(),
                                      )
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 16),
                                         nn.Linear(16, 8),
                                       nn.LayerNorm(8),
                                       nn.ReLU(),
                                      )
        
        self.head = nn.Linear(128 + 32 + 8, 140)
                   
    def forward(self, X, y=None):
        
        from_numpy = False
        
      ##
        if isinstance(X, np.ndarray):
            X = torch.from_numpy(X)
            from_numpy = True
        X = X.to(device)  # Move the input to the appropriate device if necessary
        ##
        X_256 = self.layer_seq_256(X)
        X_64 = self.layer_seq_64(X_256)
        X_8 = self.layer_seq_8(X_64)
        
        X = torch.cat([X_256, X_64, X_8], axis = 1)
        out = self.head(X)
        
        if from_numpy:
            out = out.cpu().detach().numpy()
            
        return out

In [14]:
class CiteModel_mish(nn.Module):
    
    def __init__(self, feature_num):
        super(CiteModel_mish, self).__init__()
        
        self.layer_seq_256 = nn.Sequential(nn.Linear(feature_num, 256),
                                           nn.Linear(256, 128),
                                       nn.LayerNorm(128),
                                       nn.Mish(),
                                      )
        self.layer_seq_64 = nn.Sequential(nn.Linear(128, 64),
                                       nn.Linear(64, 32),
                                       nn.LayerNorm(32),
                                       nn.Mish(),
                                      )
        self.layer_seq_8 = nn.Sequential(nn.Linear(32, 16),
                                         nn.Linear(16, 8),
                                       nn.LayerNorm(8),
                                       nn.Mish(),
                                      )
        
        self.head = nn.Linear(128 + 32 + 8, 140)
                   
    def forward(self, X, y=None):
    
        X_256 = self.layer_seq_256(X)
        X_64 = self.layer_seq_64(X_256)
        X_8 = self.layer_seq_8(X_64)
        
        X = torch.cat([X_256, X_64, X_8], axis = 1)
        out = self.head(X)
        
        return out

In [15]:
def train_loop(model, optimizer, loader, epoch):
    
    losses, lrs = [], []
    model.train()
    optimizer.zero_grad()
    #loss_fn = nn.MSELoss()
    
    with tqdm(total=len(loader),unit="batch") as pbar:
        pbar.set_description(f"Epoch{epoch}")
        
        for d in loader:
            X = d['X'].to(device)
            y = d['y'].to(device)
            
            logits = model(X)
            loss = correl_loss(logits, y)
            #loss = torch.sqrt(loss_fn(logits, y))
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            pbar.set_postfix({"loss":loss.item()})
            pbar.update(1)

    return model

In [16]:
def valid_loop(model, loader, y_val):
    
    model.eval()
    partial_correlation_scores = []
    oof_pred = []
    
    for d in loader:
        with torch.no_grad():
            val_X = d['X'].to(device).float()
            val_y = d['y'].to(device)
            logits = model(val_X)
            oof_pred.append(logits)
    
    #print(torch.cat(oof_pred).shape, torch.cat(oof_pred).detach().cpu().numpy().shape)
    cor = partial_correlation_score_torch_faster(torch.tensor(y_val).to(device), torch.cat(oof_pred))
    cor = cor.mean().item()
    logits = torch.cat(oof_pred).detach().cpu().numpy()
    
    return logits, cor

In [17]:
def test_loop(model, loader):
    
    model.eval()
    predicts=[]

    for d in tqdm(loader):
        with torch.no_grad():
            X = d['X'].to(device)
            logits = model(X)
            predicts.append(logits.detach().cpu().numpy())
            
    return np.concatenate(predicts)

### pred

In [21]:
# model #16: cite_mlp_corr_svd_128_flg_donor_val_30
pred_16 = np.zeros([48203, 140])

i = 'cite_mlp_corr_svd_128_flg_donor_val_30'
        
test_file = model_feat_dict[i][0]
# test_weight = model_feat_dict[i][1]
X_test = pd.read_pickle(cite_feature_path  + test_file)
X_test = np.array(X_test)
feature_dims = X_test.shape[1]

test_ds = CiteDataset_test(X_test)
test_dataloader = DataLoader(test_ds, batch_size=128, pin_memory=True, 
                              shuffle=False, drop_last=False, num_workers=4)

if 'mish' in i:
    model = CiteModel_mish(feature_dims)
else:
    model = CiteModel(feature_dims)
    
model = model.to(device)
model.load_state_dict(torch.load(f'{cite_mlp_path}/{i}'))

pred_16 = test_loop(model, test_dataloader).astype(np.float32)

torch.cuda.empty_cache()
        
pd.DataFrame(pred_16)   # double check train_cite_targets.h5  -> omnipath

  0%|          | 0/377 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,-1.066433,-1.500550,-1.354774,1.106190,0.869803,3.481040,5.067837,-1.570396,-1.444474,-1.320852,-1.612052,-1.534098,-1.428519,-1.496802,3.095008,-1.384286,2.648872,1.938160,-0.093022,-1.635664,-1.428120,-0.072564,-1.464633,-1.333522,4.528148,-1.577123,-1.570215,-1.564498,-1.633450,-1.551511,-1.606082,-1.488960,-1.542044,-1.542291,-1.515924,-1.584970,-1.483199,6.078822,-1.523234,-1.523154,-1.628240,-1.568818,-1.554284,-1.576391,-1.607019,-1.563618,-1.300677,-1.282699,4.529342,-1.497951,-1.504462,-1.616715,-1.604992,-1.590265,-0.913128,-0.373880,-1.571010,1.285669,-1.561719,-1.562544,-1.531397,-1.546362,-1.493735,-1.614805,-1.581547,-1.436415,-1.352806,-1.433268,0.004768,-1.507400,-1.582382,-1.650515,-1.553527,5.308838,-1.614917,2.937888,-1.565187,-0.132013,-1.512913,-1.525913,-0.179205,-1.617044,-1.141805,-1.497630,-1.589817,-1.619898,-1.517949,-1.527373,-1.324593,-1.536239,-1.349147,-1.607809,-1.596236,-1.449520,-0.069496,-1.395786,-1.459246,2.709844,-1.504532,2.281004,1.127239,-1.582563,1.390550,-1.485366,-0.814275,-1.483867,1.818746,-1.166362,3.551317,-1.324322,-1.333104,-0.007823,-1.370018,-1.791438,-1.608289,-0.100262,-0.915906,-1.476855,-1.569585,4.021511,-1.475899,0.135406,-1.601376,-1.499214,-1.508077,-1.504729,-1.617144,-1.285858,-1.497512,-1.602304,-1.585749,2.483784,-1.404202,-1.656756,-1.628249,-1.330432,-1.008113,-1.255932,-0.622936,-0.130905
1,-1.067110,-1.506955,-1.399289,1.075526,0.997081,3.506622,5.481764,-1.564466,-1.460833,-1.301538,-1.630497,-1.572419,-1.490347,-1.531179,3.168335,-1.427267,2.470253,2.130677,-0.441134,-1.637434,-1.472109,-0.058242,-1.472134,-1.346540,4.738451,-1.580705,-1.617835,-1.605957,-1.660571,-1.576319,-1.645333,-1.515913,-1.583853,-1.545092,-1.542181,-1.602096,-1.507277,6.134613,-1.544582,-1.598276,-1.658122,-1.569817,-1.591216,-1.618025,-1.622394,-1.573258,-1.365324,-1.374641,4.661467,-1.514943,-1.525232,-1.650555,-1.612984,-1.614313,-1.050634,-0.368892,-1.596666,0.777737,-1.589263,-1.581116,-1.578929,-1.556408,-1.516185,-1.611275,-1.630935,-1.470598,-1.411295,-1.485477,-0.118224,-1.505585,-1.588891,-1.651967,-1.588987,5.179552,-1.639723,2.675957,-1.584337,-0.139389,-1.552935,-1.552968,-0.562470,-1.613981,-1.148527,-1.525604,-1.601094,-1.632247,-1.561806,-1.547703,-1.396585,-1.549064,-1.390192,-1.631668,-1.603218,-1.467851,-0.253814,-1.409238,-1.494805,2.745513,-1.549431,2.249057,0.980224,-1.594142,1.342944,-1.517599,-0.857558,-1.512639,1.533982,-1.339978,3.737477,-1.391075,-1.371431,-0.343284,-1.380614,-1.574345,-1.623899,-0.251694,-1.224326,-1.529244,-1.581402,4.151693,-1.492653,0.829439,-1.611794,-1.553943,-1.531965,-1.514008,-1.643932,-1.266751,-1.511899,-1.586327,-1.620505,2.777343,-1.404117,-1.657573,-1.687846,-1.422966,-0.780818,-1.299758,-0.628505,-0.121618
2,-1.010656,-1.271471,-1.207577,2.074188,1.614956,2.007590,6.589465,-1.374983,-1.260541,-1.450437,-1.464440,-1.332187,-1.378543,-1.357404,4.944262,-1.135714,0.924946,1.518677,-0.864071,-1.520523,-1.308277,0.001427,-1.352170,-1.241024,4.492778,-1.413115,-1.387910,-1.474926,-1.551057,-1.376167,-1.415526,-1.233463,-1.391570,-1.319213,-1.332545,-1.431528,-1.287814,4.748064,-1.345603,-1.431643,-1.396456,-1.397756,-1.360939,-1.085002,-1.420398,-1.334369,-1.064852,-1.288315,1.083776,-1.353123,-1.357820,-1.394112,-1.357486,-1.375918,-0.872777,-0.296593,-1.410997,0.365104,-1.434169,-1.399326,-0.980195,-1.282730,-1.286808,-1.391266,-1.348446,-1.279712,-1.213837,-1.296896,0.074692,-1.323143,-1.357035,-1.430854,-1.481354,2.077808,-1.360545,1.681203,-1.313775,-0.382094,-1.374308,-1.361059,-0.

### - add cell_ids to train and test data
### - SHAP

In [ ]:
train_ids = np.load(index_path + "train_cite_raw_inputs_idxcol.npz", allow_pickle=True)
test_ids = np.load(index_path + "test_cite_raw_inputs_idxcol.npz", allow_pickle=True)

train_index = train_ids["index"]
train_column = train_ids["columns"]
test_index = test_ids["index"]
print(len(list(train_index)))
print(len(list(test_index)))
X_train_cell_ids = pd.read_pickle(cite_feature_path  + 'X_svd_128.pickle')   # = X_svd_128 in make-features second to last cell
X_train_cell_ids.index = train_index
X_train_cell_ids

70988
48203


In [33]:
# cell type from metadata
X_test_cell_ids = pd.read_pickle(cite_feature_path  + test_file)
X_test_cell_ids.index = test_index
X_test_cell_ids

,base_svd_0,base_svd_1,base_svd_2,base_svd_3,base_svd_4,base_svd_5,base_svd_6,base_svd_7,base_svd_8,base_svd_9,base_svd_10,base_svd_11,base_svd_12,base_svd_13,base_svd_14,base_svd_15,base_svd_16,base_svd_17,base_svd_18,base_svd_19,base_svd_20,base_svd_21,base_svd_22,base_svd_23,base_svd_24,base_svd_25,base_svd_26,base_svd_27,base_svd_28,base_svd_29,base_svd_30,base_svd_31,base_svd_32,base_svd_33,base_svd_34,base_svd_35,base_svd_36,base_svd_37,base_svd_38,base_svd_39,base_svd_40,base_svd_41,base_svd_42,base_svd_43,base_svd_44,base_svd_45,base_svd_46,base_svd_47,base_svd_48,base_svd_49,base_svd_50,base_svd_51,base_svd_52,base_svd_53,base_svd_54,base_svd_55,base_svd_56,base_svd_57,base_svd_58,base_svd_59,base_svd_60,base_svd_61,base_svd_62,base_svd_63,base_svd_64,base_svd_65,base_svd_66,base_svd_67,base_svd_68,base_svd_69,base_svd_70,base_svd_71,base_svd_72,base_svd_73,base_svd_74,base_svd_75,base_svd_76,base_svd_77,base_svd_78,base_svd_79,base_svd_80,base_svd_81,base_svd_82,base_svd_83,base_svd_84,base_svd_85,base_svd_86,base_svd_87,base_svd_88,base_svd_89,base_svd_90,base_svd_91,base_svd_92,base_svd_93,base_svd_94,base_svd_95,base_svd_96,base_svd_97,base_svd_98,base_svd_99,base_svd_100,base_svd_101,base_svd_102,base_svd_103,base_svd_104,base_svd_105,base_svd_106,base_svd_107,base_svd_108,base_svd_109,base_svd_110,base_svd_111,base_svd_112,base_svd_113,base_svd_114,base_svd_115,base_svd_116,base_svd_117,base_svd_118,base_svd_119,base_svd_120,base_svd_121,base_svd_122,base_svd_123,base_svd_124,base_svd_125,base_svd_126,base_svd_127,imp_0,imp_1,imp_2,imp_3,imp_4,imp_5,imp_6,imp_7,imp_8,imp_9,imp_10,imp_11,imp_12,imp_13,imp_14,imp_15,imp_16,imp_17,imp_18,imp_19,imp_20,imp_21,imp_22,imp_23,imp_24,imp_25,imp_26,imp_27,imp_28,imp_29,imp_30,imp_31,imp_32,imp_33,imp_34,imp_35,imp_36,imp_37,imp_38,imp_39,imp_40,imp_41,imp_42,imp_43,imp_44,imp_45,imp_46,imp_47,imp_48,imp_49,imp_50,imp_51,imp_52,imp_53,imp_54,imp_55,imp_56,imp_57,imp_58,imp_59,imp_60,imp_61,imp_62,imp_63,imp_64,imp_65,imp_66,imp_67,imp_68,imp_69,imp_70,imp_71,imp_72,imp_73,imp_74,imp_75,imp_76,imp_77,imp_78,imp_79,imp_80,imp_81,imp_82,imp_83
83d6659a6a32,94.056297,-7.824483,0.390148,-11.240480,1.049213,-6.360567,-0.291657,-0.807333,1.523107,4.391570,1.058334,1.107465,2.794737,0.232357,0.485993,0.610800,1.087533,-3.675157,-2.152177,0.258529,-0.241675,-0.281301,-0.892517,3.144524,-1.095826,0.484069,1.941438,-2.086771,0.126985,-0.265173,1.782843,0.401962,-0.815250,-1.178362,0.804193,-0.375622,1.732931,0.167944,-0.211067,0.322198,-0.055706,0.074955,-0.984684,0.062712,0.836429,-0.737014,-0.763474,-0.775506,-0.968059,0.333593,-0.577777,-0.923708,0.634263,0.665948,-0.189621,0.493161,-0.988103,-0.436438,0.022296,0.698116,-0.026895,-0.644371,0.149706,0.742314,0.001074,-0.240200,0.196376,-0.087378,-0.114259,0.454667,-1.015756,-0.507654,-0.075879,0.975734,0.691606,-0.203795,-0.039084,-0.425149,-0.519585,0.167085,-0.137737,-1.078830,0.527103,-0.183130,0.473650,-0.550963,-0.575732,-0.393081,1.038305,-0.292080,-0.802224,0.014176,0.436137,-0.697561,0.277782,-0.157537,-0.256823,-0.159989,0.244960,-0.068922,-0.413599,0.258720,0.399672,-0.208129,0.502469,0.384433,-0.383426,-0.506806,-0.165553,0.475413,0.240959,0.584796,-0.482804,0.413303,-0.030615,-0.264916,-0.404241,-0.620478,0.027995,-1.228871,-0.297079,-0.250295,0.024182,0.137247,-0.458344,-0.103292,-0.283405,-0.316505,0.000000,0.508287,0.000000,1.094377,0.000000,0.000000,1.294662,4.059924,2.392116,1.461449,0.0,0.508287,0.0,0.0,0.0,0.0,0.0,0.0,1.294662,1.294662,0.000000,0.000000,0.0,2.031354,1.461449,0.508287,1.940462,0.000000,0.0,2.559081,0.000000,0.000000,0.00000,0.000000,1.940462,0.000000,0.000000,0.000000,1.094377,2.865681,0.508287,1.461449,0.000000,3.337905,1.604354,2.559081,3.604731,2.609067,2.745583,2.450923,2.392116,2.114670,1.094377,0.508287,2.031354,1.840474,1.094377,0.000000,0.508287,0.000000,0.000000,5.809778,0.000000,2.191574,0.000000,0.0,3.449107,0.000000,4.843113,4.489811,4.320063,4.511869,1.094377,1.461449,1.294662,1.6043

In [34]:
metadata = pd.read_csv('/dss/dssfs02/lwp-dss-0001/pn36po/pn36po-dss-0001/di93zoj/neurips_competition_data/' + 'metadata.csv')
metadata.head()

,cell_id,day,donor,cell_type,technology
0,c2150f55becb,2,27678,HSC,citeseq
1,65b7edf8a4da,2,27678,HSC,citeseq
2,c1b26cb1057b,2,27678,EryP,citeseq
3,917168fa6f83,2,27678,NeuP,citeseq
4,2b29feeca86d,2,27678,EryP,citeseq


In [35]:
X_test_cell_ids = X_test_cell_ids.reset_index().rename(columns = {'index': 'cell_id'})
X_test_cell_ids = X_test_cell_ids.merge(metadata[['cell_id', 'cell_type']], on = 'cell_id', how = 'left')

In [36]:
X_test_cell_ids['cell_type'].value_counts()

cell_type
HSC     9451
MasP    9064
EryP    8788
NeuP    7719
MkP     4844
MoP     1215
BP       106
Name: count, dtype: int64

In [37]:
X_test_cell_ids

,cell_id,base_svd_0,base_svd_1,base_svd_2,base_svd_3,base_svd_4,base_svd_5,base_svd_6,base_svd_7,base_svd_8,base_svd_9,base_svd_10,base_svd_11,base_svd_12,base_svd_13,base_svd_14,base_svd_15,base_svd_16,base_svd_17,base_svd_18,base_svd_19,base_svd_20,base_svd_21,base_svd_22,base_svd_23,base_svd_24,base_svd_25,base_svd_26,base_svd_27,base_svd_28,base_svd_29,base_svd_30,base_svd_31,base_svd_32,base_svd_33,base_svd_34,base_svd_35,base_svd_36,base_svd_37,base_svd_38,base_svd_39,base_svd_40,base_svd_41,base_svd_42,base_svd_43,base_svd_44,base_svd_45,base_svd_46,base_svd_47,base_svd_48,base_svd_49,base_svd_50,base_svd_51,base_svd_52,base_svd_53,base_svd_54,base_svd_55,base_svd_56,base_svd_57,base_svd_58,base_svd_59,base_svd_60,base_svd_61,base_svd_62,base_svd_63,base_svd_64,base_svd_65,base_svd_66,base_svd_67,base_svd_68,base_svd_69,base_svd_70,base_svd_71,base_svd_72,base_svd_73,base_svd_74,base_svd_75,base_svd_76,base_svd_77,base_svd_78,base_svd_79,base_svd_80,base_svd_81,base_svd_82,base_svd_83,base_svd_84,base_svd_85,base_svd_86,base_svd_87,base_svd_88,base_svd_89,base_svd_90,base_svd_91,base_svd_92,base_svd_93,base_svd_94,base_svd_95,base_svd_96,base_svd_97,base_svd_98,base_svd_99,base_svd_100,base_svd_101,base_svd_102,base_svd_103,base_svd_104,base_svd_105,base_svd_106,base_svd_107,base_svd_108,base_svd_109,base_svd_110,base_svd_111,base_svd_112,base_svd_113,base_svd_114,base_svd_115,base_svd_116,base_svd_117,base_svd_118,base_svd_119,base_svd_120,base_svd_121,base_svd_122,base_svd_123,base_svd_124,base_svd_125,base_svd_126,base_svd_127,imp_0,imp_1,imp_2,imp_3,imp_4,imp_5,imp_6,imp_7,imp_8,imp_9,imp_10,imp_11,imp_12,imp_13,imp_14,imp_15,imp_16,imp_17,imp_18,imp_19,imp_20,imp_21,imp_22,imp_23,imp_24,imp_25,imp_26,imp_27,imp_28,imp_29,imp_30,imp_31,imp_32,imp_33,imp_34,imp_35,imp_36,imp_37,imp_38,imp_39,imp_40,imp_41,imp_42,imp_43,imp_44,imp_45,imp_46,imp_47,imp_48,imp_49,imp_50,imp_51,imp_52,imp_53,imp_54,imp_55,imp_56,imp_57,imp_58,imp_59,imp_60,imp_61,imp_62,imp_63,imp_64,imp_65,imp_66,imp_67,imp_68,imp_69,imp_70,imp_71,imp_72,imp_73,imp_74,imp_75,imp_76,imp_77,imp_78,imp_79,imp_80,imp_81,imp_82,imp_83,cell_type
0,83d6659a6a32,94.056297,-7.824483,0.390148,-11.240480,1.049213,-6.360567,-0.291657,-0.807333,1.523107,4.391570,1.058334,1.107465,2.794737,0.232357,0.485993,0.610800,1.087533,-3.675157,-2.152177,0.258529,-0.241675,-0.281301,-0.892517,3.144524,-1.095826,0.484069,1.941438,-2.086771,0.126985,-0.265173,1.782843,0.401962,-0.815250,-1.178362,0.804193,-0.375622,1.732931,0.167944,-0.211067,0.322198,-0.055706,0.074955,-0.984684,0.062712,0.836429,-0.737014,-0.763474,-0.775506,-0.968059,0.333593,-0.577777,-0.923708,0.634263,0.665948,-0.189621,0.493161,-0.988103,-0.436438,0.022296,0.698116,-0.026895,-0.644371,0.149706,0.742314,0.001074,-0.240200,0.196376,-0.087378,-0.114259,0.454667,-1.015756,-0.507654,-0.075879,0.975734,0.691606,-0.203795,-0.039084,-0.425149,-0.519585,0.167085,-0.137737,-1.078830,0.527103,-0.183130,0.473650,-0.550963,-0.575732,-0.393081,1.038305,-0.292080,-0.802224,0.014176,0.436137,-0.697561,0.277782,-0.157537,-0.256823,-0.159989,0.244960,-0.068922,-0.413599,0.258720,0.399672,-0.208129,0.502469,0.384433,-0.383426,-0.506806,-0.165553,0.475413,0.240959,0.584796,-0.482804,0.413303,-0.030615,-0.264916,-0.404241,-0.620478,0.027995,-1.228871,-0.297079,-0.250295,0.024182,0.137247,-0.458344,-0.103292,-0.283405,-0.316505,0.000000,0.508287,0.000000,1.094377,0.000000,0.000000,1.294662,4.059924,2.392116,1.461449,0.0,0.508287,0.0,0.0,0.0,0.0,0.0,0.0,1.294662,1.294662,0.000000,0.000000,0.0,2.031354,1.461449,0.508287,1.940462,0.000000,0.0,2.559081,0.000000,0.000000,0.00000,0.000000,1.940462,0.000000,0.000000,0.000000,1.094377,2.865681,0.508287,1.461449,0.000000,3.337905,1.604354,2.559081,3.604731,2.609067,2.745583,2.450923,2.392116,2.114670,1.094377,0.508287,2.031354,1.840474,1.094377,0.000000,0.508287,0.000000,0.000000,5.809778,0.000000,2.191574,0.000000,0.0,3.449107,0.000000,4.843113,4.489811,4.320063,4.511869,1.094377,1.46

In [128]:
samples_per_cell_type = 5

grouped = X_test_cell_ids.groupby('cell_type')

X_test_shap = pd.DataFrame()

# Iterate over each group (cell_type)
for cell_type, group in grouped:
    sampled_rows = group.sample(n=samples_per_cell_type, replace=False)
#     X_test_shap = X_test_shap.append(sampled_rows)   # deprecated
    X_test_shap = pd.concat([X_test_shap, sampled_rows])

X_test_shap = X_test_shap.reset_index(drop=True)
print(X_test_shap.shape)
X_test_shap.head()

(35, 214)


,cell_id,base_svd_0,base_svd_1,base_svd_2,base_svd_3,base_svd_4,base_svd_5,base_svd_6,base_svd_7,base_svd_8,base_svd_9,base_svd_10,base_svd_11,base_svd_12,base_svd_13,base_svd_14,base_svd_15,base_svd_16,base_svd_17,base_svd_18,base_svd_19,base_svd_20,base_svd_21,base_svd_22,base_svd_23,base_svd_24,base_svd_25,base_svd_26,base_svd_27,base_svd_28,base_svd_29,base_svd_30,base_svd_31,base_svd_32,base_svd_33,base_svd_34,base_svd_35,base_svd_36,base_svd_37,base_svd_38,base_svd_39,base_svd_40,base_svd_41,base_svd_42,base_svd_43,base_svd_44,base_svd_45,base_svd_46,base_svd_47,base_svd_48,base_svd_49,base_svd_50,base_svd_51,base_svd_52,base_svd_53,base_svd_54,base_svd_55,base_svd_56,base_svd_57,base_svd_58,base_svd_59,base_svd_60,base_svd_61,base_svd_62,base_svd_63,base_svd_64,base_svd_65,base_svd_66,base_svd_67,base_svd_68,base_svd_69,base_svd_70,base_svd_71,base_svd_72,base_svd_73,base_svd_74,base_svd_75,base_svd_76,base_svd_77,base_svd_78,base_svd_79,base_svd_80,base_svd_81,base_svd_82,base_svd_83,base_svd_84,base_svd_85,base_svd_86,base_svd_87,base_svd_88,base_svd_89,base_svd_90,base_svd_91,base_svd_92,base_svd_93,base_svd_94,base_svd_95,base_svd_96,base_svd_97,base_svd_98,base_svd_99,base_svd_100,base_svd_101,base_svd_102,base_svd_103,base_svd_104,base_svd_105,base_svd_106,base_svd_107,base_svd_108,base_svd_109,base_svd_110,base_svd_111,base_svd_112,base_svd_113,base_svd_114,base_svd_115,base_svd_116,base_svd_117,base_svd_118,base_svd_119,base_svd_120,base_svd_121,base_svd_122,base_svd_123,base_svd_124,base_svd_125,base_svd_126,base_svd_127,imp_0,imp_1,imp_2,imp_3,imp_4,imp_5,imp_6,imp_7,imp_8,imp_9,imp_10,imp_11,imp_12,imp_13,imp_14,imp_15,imp_16,imp_17,imp_18,imp_19,imp_20,imp_21,imp_22,imp_23,imp_24,imp_25,imp_26,imp_27,imp_28,imp_29,imp_30,imp_31,imp_32,imp_33,imp_34,imp_35,imp_36,imp_37,imp_38,imp_39,imp_40,imp_41,imp_42,imp_43,imp_44,imp_45,imp_46,imp_47,imp_48,imp_49,imp_50,imp_51,imp_52,imp_53,imp_54,imp_55,imp_56,imp_57,imp_58,imp_59,imp_60,imp_61,imp_62,imp_63,imp_64,imp_65,imp_66,imp_67,imp_68,imp_69,imp_70,imp_71,imp_72,imp_73,imp_74,imp_75,imp_76,imp_77,imp_78,imp_79,imp_80,imp_81,imp_82,imp_83,cell_type
0,6394c11656fe,91.398605,-2.573413,5.897174,0.835887,-4.996771,2.661109,-1.900818,-4.152645,1.443682,-3.602244,-1.747702,3.906802,1.733961,-2.119727,-1.108597,-3.133109,-4.540565,1.575497,-0.689293,-2.578106,2.507420,-1.177895,-1.675395,-1.366361,-1.123123,1.371597,1.645501,1.258302,-0.117805,-0.994187,-0.609635,1.246436,3.170980,-2.225565,-0.601513,-0.493538,-1.269298,0.059885,-1.085594,-0.917217,-0.597429,-0.815437,-0.426063,-1.677097,0.761961,-0.078423,-0.310130,0.488922,-0.610098,-0.611349,0.957280,-0.177487,1.554586,-0.544638,1.786080,0.542536,-0.162515,-1.671796,0.265801,-0.201104,0.867177,0.172407,-0.740333,-0.255071,0.307698,-1.323099,0.102038,-1.199427,-0.751893,-0.150644,-0.102823,0.159292,-0.199846,0.736028,-0.043564,-1.187134,-0.230672,-0.341662,-1.036231,0.625561,-0.307961,0.372681,1.036267,0.735501,-0.019753,0.549365,-0.570004,0.642439,0.146908,0.189483,-0.222768,0.764498,-0.016893,-0.758978,-0.189126,-0.510955,0.684121,0.364464,0.000470,-0.585148,0.449317,0.443050,-0.113651,0.433871,-0.280579,-0.472669,0.045730,0.579630,-0.194653,0.448946,-0.347279,-0.437040,-0.690481,0.071805,-0.488680,0.886538,0.029418,0.208392,-0.252676,0.725922,0.491173,0.030124,0.460888,-0.105347,0.394213,0.475920,-0.369894,-0.784372,0.0,0.912006,0.556557,1.939784,0.000000,0.000000,0.000000,3.562583,1.380936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.556557,0.912006,0.0,0.000000,0.000000,0.000000,0.912006,1.380936,3.474309,0.00000,0.0,2.435820,0.000000,1.173710,0.556557,0.0,1.826578,0.000000,0.000000,1.552503,0.0,2.811675,0.0,2.368422,0.000000,4.697929,0.000000,2.218248,3.241562,1.826578,1.552503,2.855459,2.435820,0.556557,0.556557,0.556557,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,5.898230,0.556557,1.380936,0.0,0.000000,2.218248,0.556557,4.582978,4.444368,4.128130,4.381195,3.181561,3.241562,0.912006,0.912006,0.556557,0.556557,0.0

In [39]:
# rename imp_ columns to gene ids:
gene_ids = ['ENSG00000075340_ADD2', 'ENSG00000233968_AL157895.1',
        'ENSG00000029534_ANK1', 'ENSG00000135046_ANXA1',
        'ENSG00000130208_APOC1', 'ENSG00000047648_ARHGAP6',
        'ENSG00000101200_AVP', 'ENSG00000166710_B2M',
        'ENSG00000130303_BST2', 'ENSG00000172247_C1QTNF4',
        'ENSG00000170458_CD14', 'ENSG00000134061_CD180',
        'ENSG00000177455_CD19', 'ENSG00000116824_CD2',
        'ENSG00000206531_CD200R1L', 'ENSG00000012124_CD22',
        'ENSG00000272398_CD24', 'ENSG00000139193_CD27',
        'ENSG00000105383_CD33', 'ENSG00000174059_CD34',
        'ENSG00000135218_CD36', 'ENSG00000004468_CD38',
        'ENSG00000010610_CD4', 'ENSG00000026508_CD44',
        'ENSG00000117091_CD48', 'ENSG00000169442_CD52',
        'ENSG00000135404_CD63', 'ENSG00000173762_CD7',
        'ENSG00000137101_CD72', 'ENSG00000019582_CD74',
        'ENSG00000105369_CD79A', 'ENSG00000085117_CD82',
        'ENSG00000114013_CD86', 'ENSG00000010278_CD9',
        'ENSG00000002586_CD99', 'ENSG00000166091_CMTM5',
        'ENSG00000119865_CNRIP1', 'ENSG00000100368_CSF2RB',
        'ENSG00000100448_CTSG', 'ENSG00000051523_CYBA',
        'ENSG00000116675_DNAJC6', 'ENSG00000142227_EMP3',
        'ENSG00000143226_FCGR2A', 'ENSG00000167996_FTH1',
        'ENSG00000139278_GLIPR1', 'ENSG00000130755_GMFG',
        'ENSG00000169567_HINT1', 'ENSG00000206503_HLA-A',
        'ENSG00000234745_HLA-B', 'ENSG00000204287_HLA-DRA',
        'ENSG00000196126_HLA-DRB1', 'ENSG00000204592_HLA-E',
        'ENSG00000171476_HOPX', 'ENSG00000076662_ICAM3',
        'ENSG00000163565_IFI16', 'ENSG00000142089_IFITM3',
        'ENSG00000160593_JAML', 'ENSG00000055118_KCNH2',
        'ENSG00000105610_KLF1', 'ENSG00000139187_KLRG1',
        'ENSG00000133816_MICAL2', 'ENSG00000198938_MT-CO3',
        'ENSG00000107130_NCS1', 'ENSG00000090470_PDCD7',
        'ENSG00000143627_PKLR', 'ENSG00000109099_PMP22',
        'ENSG00000117450_PRDX1', 'ENSG00000112077_RHAG',
        'ENSG00000108107_RPL28', 'ENSG00000198918_RPL39',
        'ENSG00000145425_RPS3A', 'ENSG00000198034_RPS4X',
        'ENSG00000196154_S100A4', 'ENSG00000197956_S100A6',
        'ENSG00000188404_SELL', 'ENSG00000124570_SERPINB6',
        'ENSG00000235169_SMIM1', 'ENSG00000095932_SMIM24',
        'ENSG00000137642_SORL1', 'ENSG00000128040_SPINK2',
        'ENSG00000072274_TFRC', 'ENSG00000205542_TMSB4X',
        'ENSG00000133112_TPT1', 'ENSG00000026025_VIM']

In [129]:
new_columns = []
for col in X_test_shap.columns:
    if col.startswith('imp_'):
        col = gene_ids[int(col.split('_')[1])]
    new_columns.append(col)
X_test_shap.columns = new_columns
print(X_test_shap.shape)
X_test_shap.head()

(35, 214)


,cell_id,base_svd_0,base_svd_1,base_svd_2,base_svd_3,base_svd_4,base_svd_5,base_svd_6,base_svd_7,base_svd_8,base_svd_9,base_svd_10,base_svd_11,base_svd_12,base_svd_13,base_svd_14,base_svd_15,base_svd_16,base_svd_17,base_svd_18,base_svd_19,base_svd_20,base_svd_21,base_svd_22,base_svd_23,base_svd_24,base_svd_25,base_svd_26,base_svd_27,base_svd_28,base_svd_29,base_svd_30,base_svd_31,base_svd_32,base_svd_33,base_svd_34,base_svd_35,base_svd_36,base_svd_37,base_svd_38,base_svd_39,base_svd_40,base_svd_41,base_svd_42,base_svd_43,base_svd_44,base_svd_45,base_svd_46,base_svd_47,base_svd_48,base_svd_49,base_svd_50,base_svd_51,base_svd_52,base_svd_53,base_svd_54,base_svd_55,base_svd_56,base_svd_57,base_svd_58,base_svd_59,base_svd_60,base_svd_61,base_svd_62,base_svd_63,base_svd_64,base_svd_65,base_svd_66,base_svd_67,base_svd_68,base_svd_69,base_svd_70,base_svd_71,base_svd_72,base_svd_73,base_svd_74,base_svd_75,base_svd_76,base_svd_77,base_svd_78,base_svd_79,base_svd_80,base_svd_81,base_svd_82,base_svd_83,base_svd_84,base_svd_85,base_svd_86,base_svd_87,base_svd_88,base_svd_89,base_svd_90,base_svd_91,base_svd_92,base_svd_93,base_svd_94,base_svd_95,base_svd_96,base_svd_97,base_svd_98,base_svd_99,base_svd_100,base_svd_101,base_svd_102,base_svd_103,base_svd_104,base_svd_105,base_svd_106,base_svd_107,base_svd_108,base_svd_109,base_svd_110,base_svd_111,base_svd_112,base_svd_113,base_svd_114,base_svd_115,base_svd_116,base_svd_117,base_svd_118,base_svd_119,base_svd_120,base_svd_121,base_svd_122,base_svd_123,base_svd_124,base_svd_125,base_svd_126,base_svd_127,ENSG00000075340_ADD2,ENSG00000233968_AL157895.1,ENSG00000029534_ANK1,ENSG00000135046_ANXA1,ENSG00000130208_APOC1,ENSG00000047648_ARHGAP6,ENSG00000101200_AVP,ENSG00000166710_B2M,ENSG00000130303_BST2,ENSG00000172247_C1QTNF4,ENSG00000170458_CD14,ENSG00000134061_CD180,ENSG00000177455_CD19,ENSG00000116824_CD2,ENSG00000206531_CD200R1L,ENSG00000012124_CD22,ENSG00000272398_CD24,ENSG00000139193_CD27,ENSG00000105383_CD33,ENSG00000174059_CD34,ENSG00000135218_CD36,ENSG00000004468_CD38,ENSG00000010610_CD4,ENSG00000026508_CD44,ENSG00000117091_CD48,ENSG00000169442_CD52,ENSG00000135404_CD63,ENSG00000173762_CD7,ENSG00000137101_CD72,ENSG00000019582_CD74,ENSG00000105369_CD79A,ENSG00000085117_CD82,ENSG00000114013_CD86,ENSG00000010278_CD9,ENSG00000002586_CD99,ENSG00000166091_CMTM5,ENSG00000119865_CNRIP1,ENSG00000100368_CSF2RB,ENSG00000100448_CTSG,ENSG00000051523_CYBA,ENSG00000116675_DNAJC6,ENSG00000142227_EMP3,ENSG00000143226_FCGR2A,ENSG00000167996_FTH1,ENSG00000139278_GLIPR1,ENSG00000130755_GMFG,ENSG00000169567_HINT1,ENSG00000206503_HLA-A,ENSG00000234745_HLA-B,ENSG00000204287_HLA-DRA,ENSG00000196126_HLA-DRB1,ENSG00000204592_HLA-E,ENSG00000171476_HOPX,ENSG00000076662_ICAM3,ENSG00000163565_IFI16,ENSG00000142089_IFITM3,ENSG00000160593_JAML,ENSG00000055118_KCNH2,ENSG00000105610_KLF1,ENSG00000139187_KLRG1,ENSG00000133816_MICAL2,ENSG00000198938_MT-CO3,ENSG00000107130_NCS1,ENSG00000090470_PDCD7,ENSG00000143627_PKLR,ENSG00000109099_PMP22,ENSG00000117450_PRDX1,ENSG00000112077_RHAG,ENSG00000108107_RPL28,ENSG00000198918_RPL39,ENSG00000145425_RPS3A,ENSG00000198034_RPS4X,ENSG00000196154_S100A4,ENSG00000197956_S100A6,ENSG00000188404_SELL,ENSG00000124570_SERPINB6,ENSG00000235169_SMIM1,ENSG00000095932_SMIM24,ENSG00000137642_SORL1,ENSG00000128040_SPINK2,ENSG00000072274_TFRC,ENSG00000205542_TMSB4X,ENSG00000133112_TPT1,ENSG00000026025_VIM,cell_type
0,6394c11656fe,91.398605,-2.573413,5.897174,0.835887,-4.996771,2.661109,-1.900818,-4.152645,1.443682,-3.602244,-1.747702,3.906802,1.733961,-2.119727,-1.108597,-3.133109,-4.540565,1.575497,-0.689293,-2.578106,2.507420,-1.177895,-1.675395,-1.366361,-1.123123,1.371597,1.645501,1.258302,-0.117805,-0.994187,-0.609635,1.246436,3.170980,-2.225565,-0.601513,-0.493538,-1.269298,0.059885,-1.085594,-0.917217,-0.597429,-0.815437,-0.426063,-1.677097,0.761961,-0.078423,-0.310130,0.488922,-0.610098,-0.611349,0.957280,-0.177487,1.554586,-0.544638,1.786080,0.542536,-0.162515,-1.671796,0.265801,-0.201104,0

In [41]:
# X_train for model #16: 'X_svd_128.pickle'
X_train = pd.read_pickle(cite_feature_path  + 'X_svd_128.pickle')
X_train = np.array(X_train)
print('X_train: ', X_train.shape)
print('X_test: ', X_test.shape)

explainer = shap.KernelExplainer(model, shap.sample(X_train, 1000))
explainer

Using 1000 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


X_train:  (70988, 212)
X_test:  (26867, 212)


In [42]:
private_test_input.shape

(26867, 22085)

In [43]:
private_test_target.shape

(26867, 140)

In [130]:
xtest = X_test_shap#.drop(['cell_id', 'cell_type'], axis=1)

In [45]:
print(X_test_shap.shape)
X_test_shap.head()

(35, 214)


,cell_id,base_svd_0,base_svd_1,base_svd_2,base_svd_3,base_svd_4,base_svd_5,base_svd_6,base_svd_7,base_svd_8,base_svd_9,base_svd_10,base_svd_11,base_svd_12,base_svd_13,base_svd_14,base_svd_15,base_svd_16,base_svd_17,base_svd_18,base_svd_19,base_svd_20,base_svd_21,base_svd_22,base_svd_23,base_svd_24,base_svd_25,base_svd_26,base_svd_27,base_svd_28,base_svd_29,base_svd_30,base_svd_31,base_svd_32,base_svd_33,base_svd_34,base_svd_35,base_svd_36,base_svd_37,base_svd_38,base_svd_39,base_svd_40,base_svd_41,base_svd_42,base_svd_43,base_svd_44,base_svd_45,base_svd_46,base_svd_47,base_svd_48,base_svd_49,base_svd_50,base_svd_51,base_svd_52,base_svd_53,base_svd_54,base_svd_55,base_svd_56,base_svd_57,base_svd_58,base_svd_59,base_svd_60,base_svd_61,base_svd_62,base_svd_63,base_svd_64,base_svd_65,base_svd_66,base_svd_67,base_svd_68,base_svd_69,base_svd_70,base_svd_71,base_svd_72,base_svd_73,base_svd_74,base_svd_75,base_svd_76,base_svd_77,base_svd_78,base_svd_79,base_svd_80,base_svd_81,base_svd_82,base_svd_83,base_svd_84,base_svd_85,base_svd_86,base_svd_87,base_svd_88,base_svd_89,base_svd_90,base_svd_91,base_svd_92,base_svd_93,base_svd_94,base_svd_95,base_svd_96,base_svd_97,base_svd_98,base_svd_99,base_svd_100,base_svd_101,base_svd_102,base_svd_103,base_svd_104,base_svd_105,base_svd_106,base_svd_107,base_svd_108,base_svd_109,base_svd_110,base_svd_111,base_svd_112,base_svd_113,base_svd_114,base_svd_115,base_svd_116,base_svd_117,base_svd_118,base_svd_119,base_svd_120,base_svd_121,base_svd_122,base_svd_123,base_svd_124,base_svd_125,base_svd_126,base_svd_127,ENSG00000075340_ADD2,ENSG00000233968_AL157895.1,ENSG00000029534_ANK1,ENSG00000135046_ANXA1,ENSG00000130208_APOC1,ENSG00000047648_ARHGAP6,ENSG00000101200_AVP,ENSG00000166710_B2M,ENSG00000130303_BST2,ENSG00000172247_C1QTNF4,ENSG00000170458_CD14,ENSG00000134061_CD180,ENSG00000177455_CD19,ENSG00000116824_CD2,ENSG00000206531_CD200R1L,ENSG00000012124_CD22,ENSG00000272398_CD24,ENSG00000139193_CD27,ENSG00000105383_CD33,ENSG00000174059_CD34,ENSG00000135218_CD36,ENSG00000004468_CD38,ENSG00000010610_CD4,ENSG00000026508_CD44,ENSG00000117091_CD48,ENSG00000169442_CD52,ENSG00000135404_CD63,ENSG00000173762_CD7,ENSG00000137101_CD72,ENSG00000019582_CD74,ENSG00000105369_CD79A,ENSG00000085117_CD82,ENSG00000114013_CD86,ENSG00000010278_CD9,ENSG00000002586_CD99,ENSG00000166091_CMTM5,ENSG00000119865_CNRIP1,ENSG00000100368_CSF2RB,ENSG00000100448_CTSG,ENSG00000051523_CYBA,ENSG00000116675_DNAJC6,ENSG00000142227_EMP3,ENSG00000143226_FCGR2A,ENSG00000167996_FTH1,ENSG00000139278_GLIPR1,ENSG00000130755_GMFG,ENSG00000169567_HINT1,ENSG00000206503_HLA-A,ENSG00000234745_HLA-B,ENSG00000204287_HLA-DRA,ENSG00000196126_HLA-DRB1,ENSG00000204592_HLA-E,ENSG00000171476_HOPX,ENSG00000076662_ICAM3,ENSG00000163565_IFI16,ENSG00000142089_IFITM3,ENSG00000160593_JAML,ENSG00000055118_KCNH2,ENSG00000105610_KLF1,ENSG00000139187_KLRG1,ENSG00000133816_MICAL2,ENSG00000198938_MT-CO3,ENSG00000107130_NCS1,ENSG00000090470_PDCD7,ENSG00000143627_PKLR,ENSG00000109099_PMP22,ENSG00000117450_PRDX1,ENSG00000112077_RHAG,ENSG00000108107_RPL28,ENSG00000198918_RPL39,ENSG00000145425_RPS3A,ENSG00000198034_RPS4X,ENSG00000196154_S100A4,ENSG00000197956_S100A6,ENSG00000188404_SELL,ENSG00000124570_SERPINB6,ENSG00000235169_SMIM1,ENSG00000095932_SMIM24,ENSG00000137642_SORL1,ENSG00000128040_SPINK2,ENSG00000072274_TFRC,ENSG00000205542_TMSB4X,ENSG00000133112_TPT1,ENSG00000026025_VIM,cell_type
0,46a536d2aba8,92.757668,-8.058743,9.577777,-0.079485,-6.457353,1.782744,1.560264,-2.409320,-5.719316,-0.597724,-3.344079,0.261510,0.712637,-1.025209,-0.820873,2.076658,-0.614023,-2.944788,-1.847399,-2.514676,2.681781,-2.113858,0.041229,1.844948,-3.045417,1.180915,0.486827,0.915940,-1.159498,-0.189862,-1.401075,0.517407,0.519325,-2.193505,0.279780,0.861897,-0.441402,-0.099304,-0.619028,0.276481,1.756663,0.022114,1.028750,0.448088,1.031782,-0.734927,-0.772353,-1.484944,0.622118,1.198598,1.077464,1.185813,2.432313,0.742448,-0.456150,-0.554208,0.281873,-0.699673,-0.123434,-0.976555,0.201653,

In [46]:
with open('X_test_shap_16.pkl', 'wb') as f:
    pickle.dump(X_test_shap, f)

In [47]:
# features: genes and svd -> omnipath: genes
# model: mostly relying on genes or svd? -> later

In [48]:
# don't need to run again: np.load('shap_values.npy', allow_pickle=True)
# %timeit
# shap_values = explainer.shap_values(xtest, nsamples=300)  #500? 
# print(len(shap_values)) # -> 140 genes
# print(len(shap_values[0])) # -> number of samples in xtest
# print(shap_values[0].shape)

# np.save('shap_values_16.npy', np.array(shap_values, dtype=object), allow_pickle=True)

In [123]:
shap_values = np.load('shap_values_16.npy', allow_pickle=True).astype(float)

In [50]:
shap_values[0]

array([[-0.00125789, -0.0289969 ,  0.19679437, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.03272793,  0.31874283, ...,  0.        ,
        -0.00038133,  0.        ],
       [ 0.        ,  0.01561575,  0.2119803 , ...,  0.        ,
         0.        ,  0.02033714],
       ...,
       [ 0.        ,  0.14269746,  0.        , ...,  0.        ,
         0.        ,  0.00223412],
       [-0.00286447,  0.0253865 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.13020312,  0.00273106, ...,  0.        ,
         0.        ,  0.        ]])